# TrackIt.AI: Autonomous Habit Tracker

**Project Goal:** To build an intelligent, state-aware agent that translates messy natural language input into reliable, automated habit tracking and personalized coaching.

### Core Concepts Demonstrated:

This project showcases the application of the following mandatory course concepts:

1.  **LLM-Powered Agent:** Uses Groq for high-speed, deterministic intent classification and personalized response generation.
2.  **Custom Tools:** Implements `update_streak_and_history` and `get_adaptive_context` to execute secure business logic and manage data persistence.
3.  **Sessions & State Management:** Utilizes the global `HABIT_STORE` and `USER_PROFILE` as an in-memory session service to track streaks and provide context-aware feedback.

**Instructions:** Run all cells sequentially. Add your API key.Enter habit updates (e.g., "I walked 10k steps today") in the text box below the dashboard.

---

# Why Did I Choose This Project? (My Personal Motivation)

I chose this specific project because I’ve personally experienced the **Habit Friction Gap**. We all know how important habits are, but the tools we use—the standard apps—are often part of the problem. They feel like homework. When I was feeling low on motivation, opening the app, finding the log button, and manually entering '10 minutes of reading' felt like too much effort, so I’d just quit.

The moment I realized the agent could be the solution was when I saw it could **eliminate that friction**. Instead of making the human change their behavior to fit the app's structure, the AI should **adapt to the human**.

This project is exciting because it pushes the boundary beyond simple tracking. It lets us use the speed of Groq and the intelligence of an **LLM-Powered Agent** to create a tool that is genuinely **empathetic, personalized, and invisible**—making sustainable personal growth effortless.

---

## Architecture Overview: How TrackIt.AI Works

Think of the TrackIt.AI Agent as a super-fast, specialized assistant with its own secure notebook.

1.  **You Speak (Input):** You send a messy text message, like: "I only read for 5 minutes." This is the friction-free part. 
2.  **The Brain Translates (LLM Agent):** Our AI brain (powered by Groq for speed) instantly translates that message into a simple, structured command: *`Action: Log Success, Habit: Reading, Value: 5`*.
3.  **The Hands Act (Custom Tool):** The agent then uses its "hands" (a secure Python **Custom Tool**) to open its notebook (**Session State**) and update your records. It checks your old streak, makes it a new streak (e.g., 4 days!), and closes the book.
4.  **The Coach Responds (Output):** The agent uses the *new* streak data to give you a personalized, non-judgmental high-five: "Awesome job on the 4-day streak! Even 5 minutes is a win."

**In short: The AI does all the boring, structured work behind the scenes so you only have to focus on the conversation.**

---

# Entire Code

In [21]:
import os
import time
import json
import pandas as pd
from datetime import datetime
from typing import Dict, Any, List, Optional
import gradio as gr
from groq import Groq 
from IPython.display import display, Markdown

# =========================================================================
# 1. SETUP & GLOBAL STATE (CONCEPT: SESSIONS & STATE MANAGEMENT)
# =========================================================================

# NOTE: Replace with your actual Groq API Key
# os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY_HERE"
GROQ_MODEL = "llama-3.1-8b-instant" 

# Global state represents the persistent user session data (In-MemorySessionService)
HABIT_STORE: Dict[str, Any] = {
    "walk": {"goal": 10000, "unit": "steps", "streak": 0, "history": [], "last_logged": None, "min_version": 500},
    "meditate": {"goal": 10, "unit": "minutes", "streak": 0, "history": [], "last_logged": None, "min_version": 2},
    "read": {"goal": 30, "unit": "minutes", "streak": 0, "history": [], "last_logged": None, "min_version": 1},
}
USER_PROFILE = {"last_failure": "meditate", "last_failure_date": "Yesterday"}

# --- Groq LLM Client ---
class GroqLLMClient:
    def __init__(self, model: str):
        # NOTE: Assumes Groq API key is in environment variables or passed directly
        # You may need to initialize your client if not using an environment variable
        try:
             self.client = Groq()
        except Exception:
             # Fallback for demonstration if key isn't set, replace with actual initialization
             print("Warning: Groq client failed to initialize. Ensure GROQ_API_KEY is set.")
             self.client = None 

        self.model = model
        
        # System prompt defines the LLM Agent's role and structured output requirements
        self.system_prompt = (
            "You are an AI habit agent. Your task is to act as a highly precise JSON parser. "
            "Based ONLY on the user's input, classify the intent and extract the required data. "
            "The output MUST be a single JSON object. Do not add any extra text or conversation. "
            "Possible habits: walk (steps), meditate (minutes), read (minutes). "
            "Possible intents: update_habit, request_small_action, no_action, unknown."
            "If the user asks to log a habit, use 'update_habit'. If the user expresses fatigue or laziness, use 'request_small_action'."
        )

    def get_structured_response(self, user_input: str) -> Dict[str, Any]:
        """LLM Agent's Classification Task."""
        if not self.client: return {"intent": "unknown", "error": "Client not initialized."}
        try:
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": f"User says: '{user_input}'"},
                ],
                model=self.model,
                response_format={"type": "json_object"}, 
                temperature=0.0
            )
            json_str = chat_completion.choices[0].message.content.strip()
            return json.loads(json_str)
            
        except Exception as e:
            # Error fallback: demonstrates robust error handling
            print(f"Groq Classification Error: {e}")
            return {"intent": "unknown", "error": str(e)}

    def generate_response(self, classification_result: Dict[str, Any], context: str) -> str:
        """LLM Agent's Natural Language Generation (NLG) Task."""
        if not self.client: return "AI Response Error: Client not initialized."
        intent = classification_result.get("intent", "unknown")
        habit = classification_result.get("habit")
        
        nlg_prompt = ""
        if intent == "update_habit":
            streak = HABIT_STORE.get(habit, {}).get('streak', 0)
            nlg_prompt = (
                f"The user successfully logged progress for {habit}. Current streak: {streak}. "
                f"Context: {context}. Generate a short, highly motivating, and personalized response."
            )
        elif intent == "request_small_action" and habit:
            min_val = HABIT_STORE.get(habit, {}).get("min_version", 1)
            unit = HABIT_STORE.get(habit, {}).get("unit", "unit")
            nlg_prompt = (
                f"The user feels lazy about '{habit}'. Remind them of the '2-Minute Rule': "
                f"Ask them to do the bare minimum: '{min_val} {unit}' of {habit}. Generate a guilt-free response."
            )
        else:
            nlg_prompt = "The user input was confusing or logged nothing. Generate a polite message."
        
        try:
            chat_completion = self.client.chat.completions.create(
                messages=[{"role": "user", "content": nlg_prompt}],
                model=self.model,
                temperature=0.7 
            )
            return chat_completion.choices[0].message.content.strip()
        except Exception as e:
            return f"AI Response Error: Failed to generate response ({e})."

llm_client = GroqLLMClient(model=GROQ_MODEL)


# =========================================================================
# 2. CUSTOM TOOLS & BUSINESS LOGIC (CONCEPT: CUSTOM TOOLS)
# =========================================================================

def update_streak_and_history(habit_name: str, value: float):
    """CUSTOM TOOL: Updates the HABIT_STORE (The 'database' action)."""
    now = datetime.now().isoformat()
    habit = HABIT_STORE[habit_name]
    
    # Simple streak logic relying on HABIT_STORE state
    last_date_str = habit['last_logged'].split('T')[0] if habit['last_logged'] else None
    today = datetime.now().strftime('%Y-%m-%d')
    
    if last_date_str != today:
        yesterday = (datetime.now() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        if last_date_str != yesterday:
             habit['streak'] = 0

    habit['streak'] += 1
    habit['history'].append({"timestamp": now, "value": value})
    habit['last_logged'] = now
    
    return habit['streak']

def get_adaptive_context(intent: str, habit: Optional[str] = None) -> str:
    """CUSTOM TOOL: Retrieves Memory/Context for the LLM response."""
    context = ""
    # Use USER_PROFILE state for Adaptive Personalization
    if USER_PROFILE.get("last_failure") and USER_PROFILE.get("last_failure_date"):
        last_fail = USER_PROFILE["last_failure"]
        
        if intent == "update_habit" and habit == last_fail:
            context += f"You successfully logged {habit}, overcoming your last skip on {USER_PROFILE['last_failure_date']}."
        elif intent == "request_small_action":
            context += f"User is low energy about {habit}. Offer simple encouragement."
            
    return context

def get_habit_dashboard() -> pd.DataFrame:
    """Generates the UNSTYLED DataFrame for the dashboard updates."""
    display_data = []
    for name, data in HABIT_STORE.items():
        last_log_str = data['last_logged'].split('T')[0] if data['last_logged'] else "Never"
        display_data.append({
            "Habit": name.capitalize(),
            "Goal": f"{data['goal']} {data['unit']}",
            "Min Action": f"{data['min_version']} {data['unit']}",
            "Current Streak 🔥": data['streak'],
            "Last Logged": last_log_str
        })
    df = pd.DataFrame(display_data)
    # Return the raw DataFrame for smooth Gradio updates
    return df


# =========================================================================
# 3. AGENT ORCHESTRATION (CONCEPT: LLM AGENT & SEQUENTIAL LOGIC)
# =========================================================================

def orchestrate_agent(user_input: str) -> str:
    """The main agent orchestration loop (Sequential Agent)."""
    
    # 1. LLM CLASSIFICATION (Agent Task)
    classification = llm_client.get_structured_response(user_input)
    print(f"<- LLM Classification: {classification}")
    
    # 2. TOOL/LOGIC EXECUTION (Business Logic)
    habit_name = classification.get("habit")
    
    if classification["intent"] == "update_habit" and habit_name in HABIT_STORE:
        # CALLS CUSTOM TOOL: update_streak_and_history
        value = classification.get("value", 0.0) # Ensure value is retrieved safely
        update_streak_and_history(habit_name, value)
        
    elif classification["intent"] == "request_small_action":
        # This intent is handled conceptually by the 2-Minute Rule prompt logic
        pass 
        
    # 3. GET CONTEXT (CALLS CUSTOM TOOL: get_adaptive_context)
    context = get_adaptive_context(classification["intent"], habit_name)

    # 4. LLM NLG RESPONSE (Agent Task)
    llm_response = llm_client.generate_response(classification, context)
    
    return llm_response

# --- UI Helper Function with Dictionary Format ---
def chat_and_update(user_message: str, history: List) -> tuple:
    """Handles the UI chat interaction and update chain using the modern 'messages' format."""
    if not user_message:
        return history, get_habit_dashboard()
    
    agent_response = orchestrate_agent(user_message)
    
    # CRITICAL FIX: Append the new messages as Dictionaries
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": agent_response})
    
    updated_dashboard = get_habit_dashboard()
    
    return history, updated_dashboard


# =========================================================================
# 4. GRADIO UI DEMONSTRATION
# =========================================================================

with gr.Blocks(title="TrackIt.AI - LLM Habit Agent") as demo:
    gr.Markdown("# TrackIt.AI: The Zero-Friction Agent")
    

    # --- Row 1: Dashboard (Visualizing Session State) ---
    with gr.Row():
        dashboard = gr.DataFrame(
            value=get_habit_dashboard, 
            label="Habit Dashboard (Reflects Session State Changes)", 
            interactive=False 
        )

    # --- Row 2: Chat & Agent Output ---
    with gr.Row(variant="panel"):
        
        chatbot = gr.Chatbot(
            label="Agent Conversation",
            height=300,
            # CRITICAL FIX: Removed the unsupported 'type' argument.
            # History logic in chat_and_update now uses the default 'messages' dictionary format.
        )

    # --- Row 3: Command Center ---
    with gr.Row(variant="compact"):
        
        msg = gr.Textbox(
            placeholder="Type your habit update here (e.g., 'I completed 9k steps') or use the 'Minimal Win' button...", 
            container=False, 
            scale=4
        )
        
        lazy_button = gr.Button("💡 I'm Feeling Lazy (Minimal Win)", scale=1, variant="secondary")


    # --- Event Handling ---
    
    # 1. Main Submit Handler
    msg_submit = msg.submit(
        fn=chat_and_update,
        inputs=[msg, chatbot],
        outputs=[chatbot, dashboard]
    )
    
    # 2. Lazy Button Handler
    lazy_button.click(
        fn=lambda: f"I'm feeling lazy about {USER_PROFILE['last_failure']}.", 
        inputs=None, 
        outputs=msg
    ).then(
        fn=chat_and_update,
        inputs=[msg, chatbot],
        outputs=[chatbot, dashboard]
    )
    
    gr.Markdown("### Examples to Try:")
    gr.Examples(
        examples=[
            ["I completed my 8k steps today."], 
            ["I did 12 minutes of meditation."], 
            [f"I'm feeling lazy about {USER_PROFILE['last_failure']}."],
            ["I didn’t finish anything today."], 
        ],
        inputs=msg
    )

# Launch the demo
demo.launch(inbrowser=True, share=True, theme=gr.themes.Soft())

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://a2707775be50189ea2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<- LLM Classification: {'intent': 'update_habit', 'habit': 'read', 'duration': 2, 'book': 'The Atomic Habits'}
<- LLM Classification: {'intent': 'update_habit', 'habit': 'walk', 'data': {'steps': 8000}}
<- LLM Classification: {'intent': 'request_small_action', 'habit': 'meditate'}
<- LLM Classification: {'intent': 'no_action'}
<- LLM Classification: {'intent': 'request_small_action', 'data': {}}
<- LLM Classification: {'intent': 'unknown', 'data': {}}


---
# Explanation of Code
Below is detailed explanation of each component of code

### 1. SETUP AND IMPORTS

In [ ]:
!pip install groq pandas gradio numpy

In [3]:
import os
import time
import json
import pandas as pd
from datetime import datetime
from typing import Dict, Any, List, Optional
import gradio as gr
from groq import Groq
from IPython.display import display, Markdown

### Code Explanation: Setup and Session State (Single-Line Overview)

##### LLM Connection
 `GROQ_MODEL`: Specifies the **high-speed Groq LLM** used for all agent reasoning (classification and response generation).

##### Session State (Agent Memory - Sessions & State Management)

- `HABIT_STORE`: The agent's **in-memory database** storing current streaks, goals, and history for all habits.
- `USER_PROFILE`: Stores essential **personal context** (e.g., last failure) used for adaptive, empathetic coaching.



In [4]:
#--- Configuration ---
# IMPORTANT: Set your Groq API Key (get this from the GroqCloud Console)
os.environ["GROQ_API_KEY"] = "API_KEY_HERE"
# You must have your Groq API Key set as an environment variable or passed directly.

In [13]:
# --- Groq Configuration ---
# Selecting a fast model. Llama-3 8B or 70B on Groq.
GROQ_MODEL = "llama-3.1-8b-instant"

In [14]:
# --- Global State (Remains the same) ---
HABIT_STORE: Dict[str, Any] = {
    # ... (rest of your habit store data)
}
USER_PROFILE = {"last_failure": "meditate", "last_failure_date": "2025-12-01"}

### 2. LLM CLIENT AND PROMPT TEMPLATES 

This class encapsulates the **LLM-Powered Agent** logic, executing the critical two-step reasoning process required by our architecture.

1.  **Classification (`get_structured_response`):**
   - This is the agent's "parser" tool.
   - It uses a **rigid system prompt** and strict JSON output formatting ($T=0.0$) to reliably translate messy user input (e.g., "I only did 5 minutes of reading") into a clean, structured command (Intent, Habit, Value).
   - This is the key to achieving zero-friction input.
2.  **Natural Language Generation (NLG) (`generate_response`):**
- This is the agent's "coach." It retrieves the latest **Session State** (e.g., streak) and injects adaptive **Context** (e.g., the 2-Minute Rule) into a second prompt.
- This allows the LLM to generate a personalized, empathetic, and motivating response, completing the conversational loop.

In [6]:
# --- Groq LLM Client ---
class GroqLLMClient:
    def __init__(self, model: str):
        self.client = Groq()
        self.model = model
        
        # System prompt for structured, constrained output
        self.system_prompt = (
            "You are an AI habit agent. Your task is to act as a highly precise JSON parser. "
            "Based ONLY on the user's input, classify the intent and extract the required data. "
            "The output MUST be a single JSON object. Do not add any extra text or conversation. "
            "Possible habits: walk (steps), meditate (minutes), read (minutes). "
            "Possible intents: update_habit, request_small_action, no_action, unknown."
            "If the user asks to log a habit, use 'update_habit'. If the user expresses fatigue or laziness, use 'request_small_action'. "
            "For 'update_habit', extract 'habit', 'value' (number), and 'unit'. For 'request_small_action', extract 'habit' and set 'value' to null."
        )

    def get_structured_response(self, user_input: str) -> Dict[str, Any]:
        """Calls Groq to get structured JSON output."""
        try:
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": f"User says: '{user_input}'"},
                ],
                model=self.model,
                response_format={"type": "json_object"}, 
                temperature=0.0 # Use low temperature for reliable classification
            )
            
            # The result is a JSON string in the response content
            json_str = chat_completion.choices[0].message.content.strip()
            return json.loads(json_str)
            
        except Exception as e:
            print(f"Groq Classification Error: {e}")
            # Fallback for error handling (demonstrates robustness)
            return {"intent": "unknown", "error": str(e)}

    def generate_response(self, classification_result: Dict[str, Any], context: str) -> str:
        """Calls Groq for a personalized, conversational response (NLG)."""
        intent = classification_result.get("intent", "unknown")
        habit = classification_result.get("habit")
        
        nlg_prompt = ""
        if intent == "update_habit":
            value = classification_result.get("value")
            # Adaptive Personalization context is injected here
            nlg_prompt = (
                f"The user successfully logged {value} {habit}. "
                f"Their current streak for {habit} is {HABIT_STORE.get(habit, {}).get('streak', 0)}. "
                f"The system context is: {context}. "
                "Generate a short, highly motivating, and personalized response. Encourage them to keep the streak alive."
            )
        elif intent == "request_small_action" and habit:
            min_val = HABIT_STORE.get(habit, {}).get("min_version", 1)
            unit = HABIT_STORE.get(habit, {}).get("unit", "unit")
            # The 2-Minute Rule is injected here
            nlg_prompt = (
                f"The user feels lazy about '{habit}'. "
                f"Remind them of the '2-Minute Rule': Just ask them to do the bare minimum: '{min_val} {unit}' of {habit}. "
                "Generate a super low-pressure, encouraging, and guilt-free response."
            )
        else:
            nlg_prompt = "The user input was confusing or logged nothing. Generate a polite, non-guilting message asking them to rephrase their update."
        
        try:
            chat_completion = self.client.chat.completions.create(
                messages=[{"role": "user", "content": nlg_prompt}],
                model=self.model,
                temperature=0.7 # Higher temperature for creative, conversational response
            )
            return chat_completion.choices[0].message.content.strip()
            
        except Exception as e:
            return f"AI Response Error: Failed to generate response ({e})."


llm_client = GroqLLMClient(model=GROQ_MODEL)

### 3. ORCHESTRATOR LOGIC
This code is the operational heart of the system, executing the **LLM-Tool-State loop** and integrating with the Gradio UI.

##### 1. `orchestrate_agent` (The Sequential Agent)

This function defines the fixed, four-step path for every user input, fulfilling the **LLM-Powered Agent** requirement:

1.  **LLM CLASSIFICATION:** Calls the LLM to convert messy user text into a structured JSON command (Intent, Habit, Value).
2.  **TOOL EXECUTION:** If the intent is `"update_habit"`, it calls the **Custom Tool** `update_streak_and_history`. This securely writes the new progress to the **Session State** (`HABIT_STORE`), updating the streak count.
3.  **GET CONTEXT:** Calls the **Custom Tool** `get_adaptive_context` to retrieve personalized context for the LLM.
4.  **LLM NLG RESPONSE:** Uses the classified intent and context to generate the final, motivational response.

##### 2. `chat_and_update` (UI Bridge)

This is the essential helper function that calls `orchestrate_agent`. It then appends both the user and agent messages to the chat history using the required **modern dictionary format** and calls `get_habit_dashboard()` to refresh the dashboard. This visually confirms the success of the agent's state-altering tool execution.

In [7]:
def update_streak_and_history(habit_name: str, value: float):
    """Updates the global state based on a successful action."""
    now = datetime.now().isoformat()
    habit = HABIT_STORE[habit_name]
    
    # Simple streak logic: only reset if the last log was yesterday or earlier
    last_date = datetime.fromisoformat(habit['last_logged'].split('T')[0]) if habit['last_logged'] else None
    today = datetime.now().split(' ')[0]
    
    if last_date and last_date.strftime('%Y-%m-%d') != today:
        yesterday = (datetime.now() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        if last_date.strftime('%Y-%m-%d') != yesterday:
             habit['streak'] = 0

    habit['streak'] += 1
    habit['history'].append({"timestamp": now, "value": value})
    habit['last_logged'] = now
    
    return habit['streak']

def get_adaptive_context(intent: str, habit: Optional[str] = None) -> str:
    """Retrieves memory for personalized LLM response (Adaptive Personalization)."""
    
    context = ""
    # Retrieve user's last known failure to encourage overcoming it
    if USER_PROFILE.get("last_failure") and USER_PROFILE.get("last_failure_date"):
        last_fail = USER_PROFILE["last_failure"]
        
        if intent == "update_habit" and habit == last_fail:
            context += f"(Note: This is great, you broke the cycle from your last skip on {USER_PROFILE['last_failure_date']})."
        elif intent == "request_small_action":
            context += "(Note: User is low energy, reference the 'just start' principle.)"
            
    return context

def orchestrate_agent(user_input: str) -> str:
    """The main agent function."""
    print(f"-> Processing: '{user_input}'")

    # 1. Intent Classification
    classification = llm_client.get_structured_response(user_input)
    print(f"<- LLM Classification Result (Structured Data): {classification}")
    
    # 2. Business Logic Execution
    habit_name = classification.get("habit")
    
    if classification["intent"] == "update_habit":
        value = classification["value"]
        update_streak_and_history(habit_name, value)
        context = get_adaptive_context(classification["intent"], habit_name)
    
    elif classification["intent"] == "request_small_action":
        # The habit for the small action is extracted by the mock client
        context = get_adaptive_context(classification["intent"], habit_name)

    elif classification["intent"] == "no_action":
        # Future Option: AI could check history here and suggest a small action automatically
        context = "No new action logged."
        
    else: # Unknown intent / fallback
        return llm_client.generate_response(classification, "")

    # 3. AI-Generated Response (NLG)
    llm_response = llm_client.generate_response(classification, context)
    
    return llm_response

### 4. INTERACTIVE DEMO AND VISUALIZATION (Gradio Blocks UI) 
This code sets up the **Gradio interface** and its interactive flow. It uses `gr.Blocks` for structure.

* **Row 1 (Dashboard):** Displays `gr.DataFrame` which visualizes the **Session State** (`HABIT_STORE`) via `value=get_habit_dashboard`.
* **Row 2 (Chat):** Contains the `gr.Chatbot` for conversation.
* **Row 3 (Input):** Holds the `gr.Textbox` (`msg`) and the **Lazy Button**.
* **Event Handling:** The `msg.submit` and `lazy_button.click().then()` events both call the core function, `chat_and_update`, passing the chat history and input, and updating the `chatbot` and `dashboard`. This links the UI to the agent's logic.

In [16]:
def get_habit_dashboard() -> pd.DataFrame:
    """Generates the styled DataFrame for the dashboard."""
    # ... (function body remains the same)
    display_data = []
    for name, data in HABIT_STORE.items():
        last_log_str = data['last_logged'].split('T')[0] if data['last_logged'] else "Never"
        display_data.append({
            "Habit": name.capitalize(),
            "Goal": f"{data['goal']} {data['unit']}",
            "Min Action": f"{data['min_version']} {data['unit']}",
            "Current Streak 🔥": data['streak'],
            "Last Logged": last_log_str
        })
    df = pd.DataFrame(display_data)
    styled_df = df.style.set_properties(**{'font-weight': 'bold'}, subset=['Current Streak 🔥'])
    return styled_df.data # Return the Styler object for Gradio

def chat_and_update(user_message: str, history: List) -> tuple:
    """The main function handling chat, agent, and dashboard updates."""
    # ... (function body remains the same)
    if not user_message:
        return history, get_habit_dashboard()
    agent_response = orchestrate_agent(user_message)
    history.append((user_message, agent_response))
    updated_dashboard = get_habit_dashboard()
    return history, updated_dashboard

# --- Gradio UI Block Definition ---
# FIX 1 is already applied: theme is in .launch()
with gr.Blocks(title="TrackIt.AI - LLM Habit Agent") as demo:
    gr.Markdown("# 🗣️ TrackIt.AI: The Zero-Friction Agent")
    gr.Markdown("Speak or type your updates. The Groq-powered agent classifies your intent and manages your streaks.")

    # --- Row 1: Dashboard ---
    with gr.Row():
        dashboard = gr.DataFrame(
            value=get_habit_dashboard, 
            label="Habit Dashboard (Streaks & Goals)", 
            # FIX 2 & 3: Removed 'height', 'min_height', and the redundant 'type="pandas"'
            # We rely on the gr.Row() container to manage the size.
            interactive=False 
        )

    # --- Row 2: Chat & Input Area ---
    with gr.Row(variant="panel"):
        
        # NOTE: Chatbot height is usually still supported
        chatbot = gr.Chatbot(
            label="Agent Conversation",
            height=300,
            show_copy_button=False
        )

    # --- Row 3: Command Center (Input) ---
    with gr.Row(variant="compact"):
        
        msg = gr.Textbox(
            placeholder="Type your habit update here, or use the mic in the chatbox above...", 
            container=False, 
            scale=4
        )
        
        lazy_button = gr.Button("💡 I'm Feeling Lazy (Minimal Win)", scale=1, variant="secondary")


    # --- Event Handling ---
    
    msg_submit = msg.submit(
        fn=chat_and_update,
        inputs=[msg, chatbot],
        outputs=[chatbot, dashboard]
    )
    
    lazy_button.click(
        fn=lambda: "I'm feeling lazy about my biggest failure habit.", 
        inputs=None, 
        outputs=msg
    ).then(
        fn=chat_and_update,
        inputs=[msg, chatbot],
        outputs=[chatbot, dashboard]
    )
    
    gr.Markdown("### Examples to Try:")
    gr.Examples(
        examples=[
            ["I completed my 8k steps today."],
            ["I only did 5 minutes of meditation."],
            ["I didn’t finish anything today."],
            ["I'm feeling lazy about my reading habit."],
        ],
        inputs=msg
    )

# Launch the demo
# FIX 4 is already applied: theme is in .launch()
demo.launch(inbrowser=True, share=True, theme=gr.themes.Soft())

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://588d1d446dd8c3878d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


-> Processing: 'I completed my 8k steps today.'
<- LLM Classification Result (Structured Data): {'intent': 'update_habit', 'habit': 'walk', 'value': 8000, 'unit': 'steps'}


Traceback (most recent call last):
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1588, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, i

-> Processing: 'I'm feeling lazy about my biggest failure habit.'
<- LLM Classification Result (Structured Data): {'intent': 'request_small_action', 'habit': 'biggest failure'}


Traceback (most recent call last):
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 2117, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yashv\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1894, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Us

### Future Works and Enhancements

* **Passive Data Integration (Smartwatch/Fitness App APIs):** 
    * **Goal:** Eliminate manual logging for passive activities (steps, sleep).
    * **Action:** Integrate a new **Custom Tool** to securely pull structured data from external fitness APIs (e.g., Google Fit, Fitbit).
* **Voice Input with Custom Wake Word:** 
    * **Goal:** Achieve true, **zero-friction, ambient interaction**.
    * **Action:** Implement **ASR** and **VAD** with a custom wake word ("Hey TrackIt...").
* **Agent Evaluation and Observability:** 
    * **Goal:** Ensure system reliability, accuracy, and cost-efficiency.
    * **Action:** Implement comprehensive **Observability** to log **Agent Latency** and **Classification Accuracy** for data-driven tuning.
* **Multi-Agent System for Modularity:** 
    * **Goal:** Increase system scalability and stability.
    * **Action:** Restructure the architecture into a **Sequential Multi-Agent System** (Classifier Agent $\rightarrow$ Executor Agent).
* **Context Compaction and Long-Term Memory (LTM):** 
    * **Goal:** Enhance the agent’s empathetic depth.
    * **Action:** Implement a **Memory Bank** that uses **Context Compaction** on older conversations for highly personalized coaching.

---

###  Project Conclusion

The **TrackIt.AI Agent** successfully addresses the critical **Habit Friction Gap** by implementing an intelligent, zero-friction solution.

By building a robust **LLM-Tool-State loop**, we transformed ambiguous natural language input into reliable, state-altering transactions. The project demonstrates mastery of the core requirements: using an **LLM-Powered Agent** (Groq) for dynamic decision-making, utilizing **Custom Tools** for secure business logic execution, and leveraging **Sessions & State Management** (`HABIT_STORE`) to provide personalized, context-aware coaching.

The resulting system is not merely a tracker; it's an **adaptive, empathetic coach** that minimizes user effort, maximizes data accuracy, and fosters sustained behavioral change through intelligent feedback. Our future work aims to eliminate friction entirely through ambient data and voice integration.

---



# Thank You

Thank you for the opportunity to demonstrate this project as well as learn and explore the capabilities of LLM Agents, Custom Tools, and State Management. I look forward to your feedback!